In [5]:
import os
from convokit import download, Corpus
import numpy as np
import pandas as pd
from sklearn.metrics import cohen_kappa_score
import matplotlib.pyplot as plt

In [2]:
wikicorpus = Corpus(filename=download("conversations-gone-awry-corpus"))
cmvcorpus = Corpus(filename=download("conversations-gone-awry-cmv-corpus"))

Dataset already exists at /home/sqt2/.convokit/downloads/conversations-gone-awry-corpus
Dataset already exists at /home/sqt2/.convokit/downloads/conversations-gone-awry-cmv-corpus


In [39]:
def get_preds(pred_path):
    pred_file = open(pred_path, 'r')
    pred_lines = pred_file.readlines()[1:]
    pred_dict = {}
    for line in pred_lines:
        id2pred = line.split(",")
        
        if len(id2pred) == 3:
            utt_id = id2pred[0]
            utt_pred = id2pred[1]
            pred_dict[utt_id] = int(utt_pred)
        elif len(id2pred) == 5:
            utt_id = id2pred[0]
            utt_pred = id2pred[4]
            pred_dict[utt_id] = int(utt_pred)
        else:
            print("ERRORRRRR")
    return pred_dict
def calculate_cohen_kappa(first, second, corpus, corpus_name):
    label_metadata = "conversation_has_personal_attack" if corpus_name == "wikiconv" else "has_removed_comment"
    first_preds = get_preds(first)
    second_preds = get_preds(second)

    first_convo_preds = []
    second_convo_preds = []
    labels = []
    for convo in corpus.iter_conversations():
        if convo.meta["split"] == "test":
            labels.append(int(convo.meta[label_metadata]))
            utt_ids = []
            for utt in convo.iter_utterances():
                utt_ids.append(utt.id)
            first_forecast_scores = [first_preds[id] for id in utt_ids if id in first_preds]
            first_convo_preds.append(max(first_forecast_scores))
            second_forecast_scores = [second_preds[id] for id in utt_ids if id in second_preds]
            second_convo_preds.append(max(second_forecast_scores))
    
    return cohen_kappa_score(first_convo_preds, second_convo_preds)
    

In [33]:
first = "/reef/sqt2/BERTCRAFT_adversarial/wikiconv/roberta-base/seed-1/predictions.csv"
second = "/reef/sqt2/SINGLE_UTT/wikiconv/roberta-base/seed-1/predictions.csv"
calculate_cohen_kappa(first,second, wikicorpus, "wikiconv")

0.47405660377358494

In [24]:
first = "/reef/sqt2/BERTCRAFT_adversarial/wikiconv/roberta-base/seed-3/predictions.csv"
second = "/reef/sqt2/SINGLE_UTT/wikiconv/roberta-base/seed-1/predictions.csv"
calculate_cohen_kappa(first,second, wikicorpus, "wikiconv")

0.5094833224329627

In [25]:
first = "/reef/sqt2/BERTCRAFT_adversarial/wikiconv/roberta-base/seed-3/predictions.csv"
second = "/reef/sqt2/SINGLE_UTT/wikiconv/roberta-base/seed-2/predictions.csv"
calculate_cohen_kappa(first,second, wikicorpus, "wikiconv")

0.5096212138222151

In [26]:
first = "/reef/sqt2/BERTCRAFT_adversarial/wikiconv/roberta-base/seed-3/predictions.csv"
second = "/reef/sqt2/SINGLE_UTT/wikiconv/roberta-base/seed-3/predictions.csv"
calculate_cohen_kappa(first,second, wikicorpus, "wikiconv")

0.5150004573310162

In [27]:
first = "/reef/sqt2/SINGLE_UTT/wikiconv/roberta-base/seed-2/predictions.csv"
second = "/reef/sqt2/SINGLE_UTT/wikiconv/roberta-base/seed-1/predictions.csv"
calculate_cohen_kappa(first,second, wikicorpus, "wikiconv")

0.5536267823930564

In [47]:
first = "/reef/sqt2/BERTCRAFT_counterfactual/wikiconv/roberta-base/seed-3/counterfactual_final.csv"
second = "/reef/sqt2/SINGLE_UTT/wikiconv/roberta-base/seed-1/predictions.csv"
calculate_cohen_kappa(first,second, wikicorpus, "wikiconv")

0.29605263157894735